In [ ]:
import tensorflow as tf
import eipy.deep_ei as d
import numpy as np
import importlib
from scikeras.wrappers import KerasClassifier

In [ ]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
inanimate_labels  = [0,1,8,9]
mask = np.isin(y_train,inanimate_labels)
y_train[mask] = 0
y_train[~mask] = 1

In [ ]:
mask = np.isin(y_test,inanimate_labels)
y_test[mask] = 0
y_test[~mask] = 1

In [ ]:
import pickle as pkl
with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
    X_train_resized=pkl.load(file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
    X_test_resized=pkl.load(file=file)
file.close()

In [ ]:
X_train_resized.shape

In [ ]:
X_sample = X_train_resized[0:8]
X_sample.shape

In [ ]:
y_sample = y_train[0:4]
y_sample

In [46]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
averaging_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(averaging_layer)

res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
net_model = Model(inputs=base_model.input, outputs=predictions)

net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
base_predictors = {
    "ResNet50" : KerasClassifier(res_model),
    "MobileNet" : KerasClassifier(net_model)
}

In [54]:
base_predictors["ResNet50"].model

In [50]:
res_model.layers[0].name

'input_3'

In [51]:
importlib.reload(d)

<module 'eipy.deep_ei' from '/home/opc/eipy/eipy/deep_ei.py'>

In [52]:
EI = d.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=2,
                        k_inner=2,
                        n_samples=1,
                        sampling_aggregation=None,
                        sampling_strategy=None,
                        n_jobs=1,# -1 could be messing up for one bp
                        metrics=metrics,
                        random_state=38,
                        project_name="images",
                        model_building=True,
                        )

In [53]:
EI.fit_base(X_sample, y_train[0:8], modality_name="images")

Training base predictors on images...
        
... for ensemble performance analysis...


INFO:tensorflow:Assets written to: ram://67ac2ed5c27344b98373607603d56c47/assets


INFO:tensorflow:Assets written to: ram://67ac2ed5c27344b98373607603d56c47/assets
2023-11-29 16:23:35.808183: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://f4a6b98f888c4c9695ad7f1985e7c4c8: INVALID_ARGUMENT: ram://f4a6b98f888c4c9695ad7f1985e7c4c8 is a directory.


1/1 [==============================] - 1s 670ms/step


INFO:tensorflow:Assets written to: ram://6a4fa6f2d39141ff9b238df6ae669df6/assets


...

INFO:tensorflow:Assets written to: ram://3f8e5e4caa7a420ba54f29e927f0692e/assets


INFO:tensorflow:Assets written to: ram://3f8e5e4caa7a420ba54f29e927f0692e/assets
2023-11-29 16:27:38.025737: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://200586e8e5234f65a422aa6d1096fd16: INVALID_ARGUMENT: ram://200586e8e5234f65a422aa6d1096fd16 is a directory.


1/1 [==============================] - 1s 720ms/step - loss: 0.6891 - accuracy: 0.5000


INFO:tensorflow:Assets written to: ram://332fc7e636224eaaac66fb747938d8bf/assets


INFO:tensorflow:Assets written to: ram://332fc7e636224eaaac66fb747938d8bf/assets
Training final base predictors: |██████████|100%

In [55]:
EI.base_summary["metrics"]

modality          images          
base predictor MobileNet  ResNet50
f_max           0.888889  0.750000
auc             0.750000  0.687500
mcc             0.000000  0.258199

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
from eipy.ei import EnsembleIntegration
from eipy.additional_ensembles import MeanAggregation, CES

In [ ]:
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    # 'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

In [ ]:
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

In [ ]:
EI.ensemble_summary["metrics"]

In [ ]:
X_test_dict = {"images": X_test_resized}

In [ ]:
y_pred = EI.predict(X_dict=X_test_dict, ensemble_model_key="S.GB")

In [ ]:
roc_auc_score(y_true=y_test, y_score=y_pred)

xor gate as nn and xnor gate as nn have same architecture. if we're training on one data set (x + labels) and testing on the other, the performance should be 0% if the instances are different and mixed if the same.